In [31]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [32]:
nlp = spacy.load('en_core_web_sm')

In [33]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [34]:
keyphrase_extractor = TextRank4Keyword()

In [35]:
#text from article https://www.gizguide.com/2020/04/apple-iphone-se-2020-leaks.html
text = """The budget iPhone is coming!
According to the report, the new iPhone will simply be dubbed as the iPhone SE like the iPhone SE in 2016. But of course, it'll feature an improved design and better internals.

The 2020 iPhone SE is expected to come with the same iPhone 8 design in White, Black, and (PRODUCT)RED colors and a 4.7-inch display, but with no 3D Touch.

Inside, the handset is expected to feature the same A13 Bionic Chip with the third generation Neural Engine. It'll able the new SE to enjoy Apple Arcade and augmented reality features.

The new iPhone might come in 3 storage options: 64GB, 128GB, and 256GB. 9to5Mac also reported that there will be 5 iPhone SE cases that will be available soon.

Reliable supply chain analyst Ming-Chi Kuo stated that the device might come with a USD 399 or around PHP 20K starting price.

If true, that price point is very attractive to those who want an ultra-fast device with longer future updates for less.

It could be the iPhone to get coming from the iPhone 6 who no longer receive major software updates.

Apple's website also leaked the Belkin InvisiGlass Ultra Screen Protector for the iPhone SE, 8, and 7.

Sources: 9to5Mac, Apple


Read more: Apple is expected to launch a budget iPhone soon! https://www.gizguide.com/2020/04/apple-iphone-se-2020-leaks.html#ixzz6JRjDjngJ
Follow us: @gizguideph on Twitter | gizguideph on Facebook"""
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN',"ADP"], window_size=8, lower=False)
tr4w.get_keywords(15)# Get top 15 keywords rankings 

iPhone - 3.1491718548495156
SE - 2.463856859693878
device - 1.6973766585497838
design - 1.5354016235827666
Apple - 1.4952871890589572
updates - 1.0777864437229436
White - 1.0302698973922904
Black - 1.0302698973922904
PRODUCT)RED - 1.0302698973922904
colors - 1.0302698973922904
display - 1.0302698973922904
chain - 1.0230685538419915
analyst - 1.0230685538419915
Ming - 1.0230685538419915
Chi - 1.0230685538419915
Kuo - 1.0230685538419915
USD - 1.0230685538419915


In [36]:
#Another article example
text_1="""Taking a look back at another week of news and headlines from Cupertino, this week’s Apple Loop includes all you need to know about the new iPhone SE, the stunning camera tech in the iPhone 12, Apple shuts down Dark Sky on Android, Apple’s AirTag video leak, Intel’s new chips for the MacBook Pro, and the best peripheral for Working From Home…

Apple Loop is here to remind you of a few of the very many discussions that have happened around Apple over the last seven days (and you can read my weekly digest of Android news here on Forbes).

Here Comes The iPhone SE, 2020 Edition!

Apple’s next handset launch is drawing ever closer. The long rumoured replacement for the iPhone SE is expected to sport the same chipset as the iPhone 11, but in a smaller frame… likely matching the design of the iPhone 8. The team at 9to5Mac have learned not only the new handset’s name, but the storage and color options. Let’s meet the iPhone SE (2020):

“…Apple will simply call the new entry-level model “iPhone SE” while referencing the new hardware as the 2020 version. Apple originally used the iPhone SE marketing name for the 4-inch model that use the iPhone 5s body with the camera and processor from the iPhone 6s.”

Zac Hall and the 9to5 team are also ‘confirming’ the color choices (white, black, and red), storage options (64 GB, 128 GB, or 256 GB), and the first-party cases (black or white silicone; red, black, or midnight blue leather).

As for the launch date, while it may come sooner than some think. Although some retailers are preparing to put third-party cases on sale next week, many are looking at April 15th as the launch date, with availability on the 22nd. It may be a cheaper handset (with an expected entry-level price of $399) but it is a critical handset for Apple’s 2020:

“The iPhone 9 is going to be an important handset for Apple even though many believe the original launch plans have been disrupted. As Tim Cook and his team push the main iPhone line higher up the price bands, with handsets touching $1500 dollars, this pull has resulted in a price gap underneath the new handsets. That gap is filling up with ‘near flagship’ Android devices. Apple is losing the mid-tier consumers as it chases the top of the market.

“Hence the iPhone 9, with its rumored $399 price point, becomes a key handset, blocking the flow of users to Android and bringing them into Apple’s cloud based services with a wealth of subscription services.”

More about the impact of the new iPhone here on Forbes.

Apple Debuts Latest Products
CUPERTINO, CALIFORNIA – SEPTEMBER 12: Phil Schiller, senior vice president of worldwide marketing at … [+] Apple Inc., speaks at an Apple event at the Steve Jobs Theater at Apple Park on September 12, 2018 in Cupertino, California. Apple is expected to announce new iPhones with larger screens as well as other product upgrades. (Photo by Justin Sullivan/Getty Images)

Getty Images

More iPhone 12 Camera Advances

The recently launched iPad Pro may hold one of the secrets to the iPhone 12’s camera. The LiDAR scanner inside the tablet is also the perfect size to fit in the iPhone 11 Pro camera island. With new stabilisation on the sensor, faster hardware, and improved software, Apple is going to be using imaging as a major selling point of the new flagship. Forbes’ Gordon Kelly reports:

“The downside of this discovery is it strongly suggests Apple will not be making dramatic changes to the iPhone 11 Pro and Pro Max’s camera hardware for the iPhone 12. But that is unsurprising given the massive step-up in quality these models made, from the iPhone XS and XS Max. The flip side, looking at the early demos from developers (see below), is the LiDAR scanner looks set to be a true differential against rivals and a genuine game-changer.”

Read more on the new features here.

Apple Buys Dark Sky, Prepares To Shut Out Android Users

Apple made a rare public purchase this week. It now has the popular weather app Dark Sky. But it comes with a bit of a shock to the community. Previously a cross-platform app, Dark Sky can no longer be downloaded by Android or WearOS users. Existing users will see the app shut down on July 1st 2020. Apps that use the API have a little longer, with existing access available until the end of 2021, but no new sign-ups will be allowed. Arol Wright looks at the impact:

“Despite how invested Apple is with their ecosystem of apps, they don’t normally do this. Shazam, for example, was acquired by Apple in 2018, yet they still keep an Android version of the app active. One of the possible following steps for Apple, then, is that they might roll it into iOS natively at some point, merging it with the existing Weather app. Web services are also being gutted together with the Android app.”

More thoughts on the move at XDA Developers and elsewhere.

Apple Leaks AirTag Details

The oft-mentioned but still to be launched AirTag tracking hardware has shown up in a new video posted to Apple’s YouTube channel. Chance Miller looks at the details from the (now removed) video:

“Apple mentions AirTags in reference to the Find My app’s offline finding feature:

Offline finding enables this device and AirTags to be found when not connected to Wi-Fi or cellular.

The Apple tracker tags will allow customers to locate real-world objects — like keys or suitcases — in the Find My app. As Apple’s video teases, the tags will use the offline features introduced in iOS 13 to allow anyone’s device to be able to detect the tracker signal.”

More at 9to5Mac.

New Chips For The MacBook Pro?

This week saw Intel announce the latest 10th-generation Core processors. Designed for more powerful machines, a number of the geekerati are wondering when these will end up in Apple’s MacBook Pro machines. Joe Rossignol reports:

“The new H-series chips have the same base clock speeds as the 9th-generation chips in the current 16-inch MacBook Pro, but Turbo Boost speeds now exceed 5GHz for the first time. For example, the new highest-end Core i9 chip still clocks in at 2.4GHz, but its maximum Turbo Boost frequency has increased from 5.0GHz to 5.3GHz.”

More at MacRumors.

US-IT-APPLE-NEWS-FEED
MacBook Pros are seen on display during a product launch event at Apple headquarters in Cupertino, … [+] California on October 27, 2016. – Apple revealed its new line of MacBook Pro laptops as well as a new Apple TV app. (Photo by Josh Edelson / AFP) (Photo credit should read JOSH EDELSON/AFP via Getty Images)

AFP via Getty Images

And Finally…

Working from home? Doing a lot of video conferencing? Wondering what to do with headphones and a microphone and just one 3.5mm headphone jack on your MacBook? Never fear, Adrian Kingsley-Hughes has some advice:

Well, the problem is compatibility between the microphone jack and the jack built into the MacBook. The headphone jack is called TRS, which stands for “tip, ring, sleeve,” and if you look closely at the jack, you’ll seen three bands on it. The MacBook needs a TRRS jack, which stands for “tip, ring, ring, sleeve,” and this has four bands.

So, what can you do? You have a few options. The easiest way around this is to use a converter cable like this, which will cost less than $10. Alternatively, you can get a USB-A or USB-C which will plug into a USB port, no drivers needed. Again, these will cost you less than $10.

More at ZDNet. Personally I’ve been using Sabrent’s USB adaptor that gives you a mic and a headphone jack to work with. Very much recommended.

Apple Loop brings you seven days worth of highlights every weekend here on Forbes. Don’t forget to follow me so you don’t miss any coverage in the future. Last week’s Apple Loop can be read here, or this week’s edition of Loop’s sister column, Android Circuit, is also available on Forbes."""
tr4w = TextRank4Keyword()
tr4w.analyze(text_1, candidate_pos = ['NOUN', 'PROPN',"ADP"], window_size=8, lower=False)
tr4w.get_keywords(15)#getting top 15 keywords

Apple - 11.103266218935733
iPhone - 6.029632850957831
MacBook - 3.703519472298296
Pro - 3.444164018734229
Android - 3.3586225588379746
video - 3.077677500522632
week - 2.672864784686987
camera - 2.4018679354165706
jack - 2.212522695990824
SE - 2.051672673692578
chips - 2.03498274460045
app - 2.0289576807365464
hardware - 1.9322202254294756
Loop - 1.9133141131634592
Cupertino - 1.8616293075685744
price - 1.778380351314779
launch - 1.7637900770946158


In [37]:
#extractive Summary
!pip install sumy

In [38]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

class TextSummary(object):

    def __init__(self, feeds_str, num_sents):
        self.summary = str()
        
        parser = PlaintextParser.from_string(feeds_str, Tokenizer("english"))
        summarizer = LexRankSummarizer()

        sentences = summarizer(parser.document, num_sents)  # Summarize the document with 5 sentences
        for sentence in sentences:
            self.summary += (sentence.__unicode__())

    def output(self):
        return self.summary

In [39]:
#text from article https://www.gizguide.com/2020/04/apple-iphone-se-2020-leaks.html
input_text = """The budget iPhone is coming!
According to the report, the new iPhone will simply be dubbed as the iPhone SE like the iPhone SE in 2016. But of course, it'll feature an improved design and better internals.

The 2020 iPhone SE is expected to come with the same iPhone 8 design in White, Black, and (PRODUCT)RED colors and a 4.7-inch display, but with no 3D Touch.

Inside, the handset is expected to feature the same A13 Bionic Chip with the third generation Neural Engine. It'll able the new SE to enjoy Apple Arcade and augmented reality features.

The new iPhone might come in 3 storage options: 64GB, 128GB, and 256GB. 9to5Mac also reported that there will be 5 iPhone SE cases that will be available soon.

Reliable supply chain analyst Ming-Chi Kuo stated that the device might come with a USD 399 or around PHP 20K starting price.

If true, that price point is very attractive to those who want an ultra-fast device with longer future updates for less.

It could be the iPhone to get coming from the iPhone 6 who no longer receive major software updates.

Apple's website also leaked the Belkin InvisiGlass Ultra Screen Protector for the iPhone SE, 8, and 7.

Sources: 9to5Mac, Apple


Read more: Apple is expected to launch a budget iPhone soon! https://www.gizguide.com/2020/04/apple-iphone-se-2020-leaks.html#ixzz6JRjDjngJ
Follow us: @gizguideph on Twitter | gizguideph on Facebook"""

In [40]:
## the following takes input as text and number of sentences we need for summary
text_to_sum = TextSummary(input_text,5)
print(text_to_sum.output())

According to the report, the new iPhone will simply be dubbed as the iPhone SE like the iPhone SE in 2016.The 2020 iPhone SE is expected to come with the same iPhone 8 design in White, Black, and (PRODUCT)RED colors and a 4.7-inch display, but with no 3D Touch.The new iPhone might come in 3 storage options: 64GB, 128GB, and 256GB.It could be the iPhone to get coming from the iPhone 6 who no longer receive major software updates.Read more: Apple is expected to launch a budget iPhone soon!


In [41]:
#Another text article from https://leadingmobilenews.com/new-iphone-se-exposed-stunning-iphone-12-leaks-apple-swiftly-shuts-down-
input_text1="""Taking a look back at another week of news and headlines from Cupertino, this week’s Apple Loop includes all you need to know about the new iPhone SE, the stunning camera tech in the iPhone 12, Apple shuts down Dark Sky on Android, Apple’s AirTag video leak, Intel’s new chips for the MacBook Pro, and the best peripheral for Working From Home…

Apple Loop is here to remind you of a few of the very many discussions that have happened around Apple over the last seven days (and you can read my weekly digest of Android news here on Forbes).

Here Comes The iPhone SE, 2020 Edition!

Apple’s next handset launch is drawing ever closer. The long rumoured replacement for the iPhone SE is expected to sport the same chipset as the iPhone 11, but in a smaller frame… likely matching the design of the iPhone 8. The team at 9to5Mac have learned not only the new handset’s name, but the storage and color options. Let’s meet the iPhone SE (2020):

“…Apple will simply call the new entry-level model “iPhone SE” while referencing the new hardware as the 2020 version. Apple originally used the iPhone SE marketing name for the 4-inch model that use the iPhone 5s body with the camera and processor from the iPhone 6s.”

Zac Hall and the 9to5 team are also ‘confirming’ the color choices (white, black, and red), storage options (64 GB, 128 GB, or 256 GB), and the first-party cases (black or white silicone; red, black, or midnight blue leather).

As for the launch date, while it may come sooner than some think. Although some retailers are preparing to put third-party cases on sale next week, many are looking at April 15th as the launch date, with availability on the 22nd. It may be a cheaper handset (with an expected entry-level price of $399) but it is a critical handset for Apple’s 2020:

“The iPhone 9 is going to be an important handset for Apple even though many believe the original launch plans have been disrupted. As Tim Cook and his team push the main iPhone line higher up the price bands, with handsets touching $1500 dollars, this pull has resulted in a price gap underneath the new handsets. That gap is filling up with ‘near flagship’ Android devices. Apple is losing the mid-tier consumers as it chases the top of the market.

“Hence the iPhone 9, with its rumored $399 price point, becomes a key handset, blocking the flow of users to Android and bringing them into Apple’s cloud based services with a wealth of subscription services.”

More about the impact of the new iPhone here on Forbes.

Apple Debuts Latest Products
CUPERTINO, CALIFORNIA – SEPTEMBER 12: Phil Schiller, senior vice president of worldwide marketing at … [+] Apple Inc., speaks at an Apple event at the Steve Jobs Theater at Apple Park on September 12, 2018 in Cupertino, California. Apple is expected to announce new iPhones with larger screens as well as other product upgrades. (Photo by Justin Sullivan/Getty Images)

Getty Images

More iPhone 12 Camera Advances

The recently launched iPad Pro may hold one of the secrets to the iPhone 12’s camera. The LiDAR scanner inside the tablet is also the perfect size to fit in the iPhone 11 Pro camera island. With new stabilisation on the sensor, faster hardware, and improved software, Apple is going to be using imaging as a major selling point of the new flagship. Forbes’ Gordon Kelly reports:

“The downside of this discovery is it strongly suggests Apple will not be making dramatic changes to the iPhone 11 Pro and Pro Max’s camera hardware for the iPhone 12. But that is unsurprising given the massive step-up in quality these models made, from the iPhone XS and XS Max. The flip side, looking at the early demos from developers (see below), is the LiDAR scanner looks set to be a true differential against rivals and a genuine game-changer.”

Read more on the new features here.

Apple Buys Dark Sky, Prepares To Shut Out Android Users

Apple made a rare public purchase this week. It now has the popular weather app Dark Sky. But it comes with a bit of a shock to the community. Previously a cross-platform app, Dark Sky can no longer be downloaded by Android or WearOS users. Existing users will see the app shut down on July 1st 2020. Apps that use the API have a little longer, with existing access available until the end of 2021, but no new sign-ups will be allowed. Arol Wright looks at the impact:

“Despite how invested Apple is with their ecosystem of apps, they don’t normally do this. Shazam, for example, was acquired by Apple in 2018, yet they still keep an Android version of the app active. One of the possible following steps for Apple, then, is that they might roll it into iOS natively at some point, merging it with the existing Weather app. Web services are also being gutted together with the Android app.”

More thoughts on the move at XDA Developers and elsewhere.

Apple Leaks AirTag Details

The oft-mentioned but still to be launched AirTag tracking hardware has shown up in a new video posted to Apple’s YouTube channel. Chance Miller looks at the details from the (now removed) video:

“Apple mentions AirTags in reference to the Find My app’s offline finding feature:

Offline finding enables this device and AirTags to be found when not connected to Wi-Fi or cellular.

The Apple tracker tags will allow customers to locate real-world objects — like keys or suitcases — in the Find My app. As Apple’s video teases, the tags will use the offline features introduced in iOS 13 to allow anyone’s device to be able to detect the tracker signal.”

More at 9to5Mac.

New Chips For The MacBook Pro?

This week saw Intel announce the latest 10th-generation Core processors. Designed for more powerful machines, a number of the geekerati are wondering when these will end up in Apple’s MacBook Pro machines. Joe Rossignol reports:

“The new H-series chips have the same base clock speeds as the 9th-generation chips in the current 16-inch MacBook Pro, but Turbo Boost speeds now exceed 5GHz for the first time. For example, the new highest-end Core i9 chip still clocks in at 2.4GHz, but its maximum Turbo Boost frequency has increased from 5.0GHz to 5.3GHz.”

More at MacRumors.

US-IT-APPLE-NEWS-FEED
MacBook Pros are seen on display during a product launch event at Apple headquarters in Cupertino, … [+] California on October 27, 2016. – Apple revealed its new line of MacBook Pro laptops as well as a new Apple TV app. (Photo by Josh Edelson / AFP) (Photo credit should read JOSH EDELSON/AFP via Getty Images)

AFP via Getty Images

And Finally…

Working from home? Doing a lot of video conferencing? Wondering what to do with headphones and a microphone and just one 3.5mm headphone jack on your MacBook? Never fear, Adrian Kingsley-Hughes has some advice:

Well, the problem is compatibility between the microphone jack and the jack built into the MacBook. The headphone jack is called TRS, which stands for “tip, ring, sleeve,” and if you look closely at the jack, you’ll seen three bands on it. The MacBook needs a TRRS jack, which stands for “tip, ring, ring, sleeve,” and this has four bands.

So, what can you do? You have a few options. The easiest way around this is to use a converter cable like this, which will cost less than $10. Alternatively, you can get a USB-A or USB-C which will plug into a USB port, no drivers needed. Again, these will cost you less than $10.

More at ZDNet. Personally I’ve been using Sabrent’s USB adaptor that gives you a mic and a headphone jack to work with. Very much recommended.

Apple Loop brings you seven days worth of highlights every weekend here on Forbes. Don’t forget to follow me so you don’t miss any coverage in the future. Last week’s Apple Loop can be read here, or this week’s edition of Loop’s sister column, Android Circuit, is also available on Forbes."""

In [42]:
# the following takes input as text and number of sentences we need for summary
text_to_sum_1 = TextSummary(input_text1,5)
print(text_to_sum_1.output())

Taking a look back at another week of news and headlines from Cupertino, this week’s Apple Loop includes all you need to know about the new iPhone SE, the stunning camera tech in the iPhone 12, Apple shuts down Dark Sky on Android, Apple’s AirTag video leak, Intel’s new chips for the MacBook Pro, and the best peripheral for Working From Home…It may be a cheaper handset (with an expected entry-level price of $399) but it is a critical handset for Apple’s 2020:More about the impact of the new iPhone here on Forbes.“The downside of this discovery is it strongly suggests Apple will not be making dramatic changes to the iPhone 11 Pro and Pro Max’s camera hardware for the iPhone 12.Chance Miller looks at the details from the (now removed) video:
